# SPR 2026 - TF-IDF + Data Augmentation + Ensemble

## ✅ Funciona 100% OFFLINE

**Este notebook combina os 3 melhores modelos com Data Augmentation:**
- LinearSVC (0.77885)
- SGDClassifier (0.75019)
- LogisticRegression (0.72935)

---
### Estratégias de Augmentation:
1. **EDA (Easy Data Augmentation)** - Random swap, delete, insert
2. **Noise Injection** - Simula erros de digitação (comum em laudos reais)
3. **Segment Shuffle** - Embaralha sentenças mantendo semântica

---
### Configuração Kaggle:
1. Settings → **Internet OFF**
2. Settings → Accelerator → **None** (não precisa GPU)
3. Run All

---

In [ ]:
# ===== SPR 2026 - TF-IDF + DATA AUGMENTATION + ENSEMBLE =====

import os
import re
import random
import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score, classification_report

print("="*60)
print("SPR 2026 - TF-IDF + Data Augmentation + Ensemble")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# Augmentation config
AUG_FACTOR = 2  # Quantas vezes multiplicar dados de classes minoritárias
MIN_SAMPLES = 500  # Mínimo de amostras por classe após augmentation

random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# ==== CARREGAR DADOS ====
print("\n[1/7] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f"    Train original: {train.shape}")
print(f"    Test: {test.shape}")

# Distribuição original
print("\nDistribuição das classes (original):")
class_dist = train['label'].value_counts().sort_index()
for label, count in class_dist.items():
    print(f"    Classe {label}: {count:5d} ({100*count/len(train):.1f}%)")

In [ ]:
# ==== FUNÇÕES DE AUGMENTATION ====
print("\n[2/7] Definindo funções de augmentation...")

# Dicionário de sinônimos médicos (mamografia/radiologia)
MEDICAL_SYNONYMS = {
    'nodulo': ['nódulo', 'lesão nodular', 'formação nodular'],
    'nódulo': ['nodulo', 'lesão nodular', 'formação nodular'],
    'mama': ['glândula mamária', 'tecido mamário'],
    'calcificação': ['microcalcificação', 'calcificações'],
    'calcificações': ['microcalcificações', 'calcificação'],
    'benigno': ['benigna', 'aspecto benigno'],
    'benigna': ['benigno', 'aspecto benigno'],
    'maligno': ['maligna', 'suspeito de malignidade'],
    'maligna': ['maligno', 'suspeito de malignidade'],
    'densas': ['densa', 'densidade aumentada'],
    'densa': ['densas', 'densidade aumentada'],
    'bilateral': ['em ambas as mamas', 'bilateralmente'],
    'unilateral': ['em uma mama', 'unilateralmente'],
    'assimetria': ['assimétrico', 'área assimétrica'],
    'contornos': ['margens', 'bordas', 'limites'],
    'margens': ['contornos', 'bordas', 'limites'],
    'espiculado': ['espiculada', 'com espículas'],
    'circunscrito': ['circunscrita', 'bem delimitado'],
    'irregular': ['irregulares', 'de contornos irregulares'],
    'regular': ['regulares', 'de contornos regulares'],
    'direita': ['D', 'dir'],
    'esquerda': ['E', 'esq'],
    'superior': ['sup', 'parte superior'],
    'inferior': ['inf', 'parte inferior'],
    'lateral': ['lat', 'região lateral'],
    'medial': ['med', 'região medial'],
    'axila': ['região axilar', 'axilar'],
    'linfonodo': ['gânglio', 'linfonodomegalia'],
    'gânglio': ['linfonodo', 'linfonodomegalia'],
    'parênquima': ['tecido', 'parênquima mamário'],
    'estroma': ['tecido conjuntivo', 'estroma mamário'],
    'cm': ['centímetros', 'centimetros'],
    'mm': ['milímetros', 'milimetros'],
    'apresenta': ['mostra', 'evidencia', 'revela'],
    'observa-se': ['nota-se', 'visualiza-se', 'identifica-se'],
    'nota-se': ['observa-se', 'visualiza-se', 'identifica-se'],
    'sem': ['ausência de', 'não há'],
    'com': ['apresentando', 'evidenciando'],
}

def tokenize(text):
    """Tokeniza texto em palavras."""
    return re.findall(r'\b\w+\b', text.lower())

def detokenize(tokens, original_text):
    """Reconstrói texto a partir de tokens (mantém pontuação original)."""
    result = original_text
    orig_tokens = re.findall(r'\b\w+\b', original_text.lower())
    
    for i, (orig, new) in enumerate(zip(orig_tokens, tokens)):
        if orig != new:
            # Preserva capitalização original
            pattern = re.compile(re.escape(orig), re.IGNORECASE)
            result = pattern.sub(new, result, count=1)
    return result

def synonym_replacement(text, n=2):
    """Substitui n palavras por sinônimos médicos."""
    words = tokenize(text)
    candidates = [(i, w) for i, w in enumerate(words) if w in MEDICAL_SYNONYMS]
    
    if not candidates:
        return text
    
    n = min(n, len(candidates))
    selected = random.sample(candidates, n)
    
    new_words = words.copy()
    for i, word in selected:
        synonyms = MEDICAL_SYNONYMS[word]
        new_words[i] = random.choice(synonyms)
    
    return ' '.join(new_words)

def random_swap(text, n=2):
    """Troca n pares de palavras aleatoriamente."""
    words = tokenize(text)
    if len(words) < 4:
        return text
    
    n = min(n, len(words) // 2)
    new_words = words.copy()
    
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(new_words)), 2)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    
    return ' '.join(new_words)

def random_deletion(text, p=0.1):
    """Deleta palavras com probabilidade p."""
    words = tokenize(text)
    if len(words) <= 5:
        return text
    
    new_words = [w for w in words if random.random() > p]
    
    # Garantir mínimo de palavras
    if len(new_words) < 3:
        return text
    
    return ' '.join(new_words)

def noise_injection(text, p=0.05):
    """Simula erros de digitação (comum em laudos reais)."""
    chars = list(text)
    typo_map = {
        'a': 'as', 'e': 'er', 'i': 'io', 'o': 'op',
        's': 'sa', 'r': 're', 'n': 'nm', 'm': 'mn',
        'c': 'cv', 'd': 'df', 'l': 'lk'
    }
    
    result = []
    for c in chars:
        if c.lower() in typo_map and random.random() < p:
            # 50% chance: duplicar letra, 50%: trocar por vizinha
            if random.random() < 0.5:
                result.append(c + c)  # duplicar
            else:
                result.append(random.choice(typo_map.get(c.lower(), c)))  # vizinha
        else:
            result.append(c)
    
    return ''.join(result)

def sentence_shuffle(text):
    """Embaralha frases do laudo (mantém semântica geral)."""
    sentences = re.split(r'[.!?]\s*', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    if len(sentences) < 2:
        return text
    
    random.shuffle(sentences)
    return '. '.join(sentences) + '.'

def eda_augment(text, alpha=0.3):
    """EDA completo: aplica todas as técnicas com probabilidade."""
    augmented = text
    
    if random.random() < alpha:
        augmented = synonym_replacement(augmented, n=random.randint(1, 3))
    
    if random.random() < alpha:
        augmented = random_swap(augmented, n=random.randint(1, 2))
    
    if random.random() < alpha * 0.5:  # Menos agressivo
        augmented = random_deletion(augmented, p=0.1)
    
    if random.random() < alpha * 0.3:  # Ainda mais conservador
        augmented = noise_injection(augmented, p=0.03)
    
    if random.random() < alpha * 0.5:
        augmented = sentence_shuffle(augmented)
    
    return augmented

# Testar
sample = train.iloc[0]['text']
print(f"Original: {sample[:100]}...")
print(f"Augmented: {eda_augment(sample)[:100]}...")

In [ ]:
# ==== APLICAR AUGMENTATION ====
print("\n[3/7] Aplicando data augmentation...")

# Calcular quantas amostras gerar por classe
class_counts = train['label'].value_counts()
max_count = class_counts.max()

augmented_data = []

for label in sorted(train['label'].unique()):
    class_data = train[train['label'] == label]
    current_count = len(class_data)
    
    # Target: mínimo entre max_count e MIN_SAMPLES * AUG_FACTOR
    target_count = max(min(max_count, current_count * AUG_FACTOR), MIN_SAMPLES)
    samples_needed = target_count - current_count
    
    if samples_needed > 0:
        # Gerar amostras augmentadas
        for _ in range(samples_needed):
            idx = random.randint(0, current_count - 1)
            original_row = class_data.iloc[idx]
            aug_text = eda_augment(original_row['text'])
            augmented_data.append({
                'id': f"aug_{original_row['id']}_{len(augmented_data)}",
                'text': aug_text,
                'label': label
            })
        print(f"    Classe {label}: {current_count} → {current_count + samples_needed} (+{samples_needed})")
    else:
        print(f"    Classe {label}: {current_count} (sem augmentation)")

# Combinar dados originais + augmentados
if augmented_data:
    aug_df = pd.DataFrame(augmented_data)
    train_aug = pd.concat([train, aug_df], ignore_index=True)
else:
    train_aug = train.copy()

print(f"\nTrain após augmentation: {len(train_aug)} ({len(train_aug) - len(train)} novas amostras)")

# Nova distribuição
print("\nNova distribuição:")
new_dist = train_aug['label'].value_counts().sort_index()
for label, count in new_dist.items():
    print(f"    Classe {label}: {count:5d} ({100*count/len(train_aug):.1f}%)")

In [ ]:
# ==== PREPROCESSAMENTO ====
print("\n[4/7] Preprocessamento de texto...")

def preprocess(text):
    """Preprocessamento leve mantendo termos médicos."""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # Remove números isolados mas mantém medidas (ex: 2,3cm)
    text = re.sub(r'\b\d{4,}\b', '', text)  # Remove IDs longos
    # Normaliza espaços
    text = re.sub(r'\s+', ' ', text).strip()
    return text

X_train_aug = train_aug['text'].apply(preprocess)
y_train_aug = train_aug['label'].values
X_test = test['text'].apply(preprocess)

print(f"    X_train_aug: {len(X_train_aug)}")
print(f"    X_test: {len(X_test)}")

In [ ]:
# ==== TF-IDF VETORIZAÇÃO ====
print("\n[5/7] Vetorização TF-IDF...")

tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    strip_accents='unicode',
)

X_train_tfidf = tfidf.fit_transform(X_train_aug)
X_test_tfidf = tfidf.transform(X_test)

print(f"    Vocabulary size: {len(tfidf.vocabulary_)}")
print(f"    Train shape: {X_train_tfidf.shape}")
print(f"    Test shape: {X_test_tfidf.shape}")

In [ ]:
# ==== TREINAR MODELOS TOP 3 ====
print("\n[6/7] Treinando Top 3 modelos...")

# 1. LinearSVC (melhor single model: 0.77885)
print("\n--- LinearSVC ---")
svc = LinearSVC(
    C=0.5,
    loss='squared_hinge',
    max_iter=3000,
    class_weight='balanced',
    dual='auto',
    random_state=SEED
)
# Calibrar para ter predict_proba
svc_calibrated = CalibratedClassifierCV(svc, cv=3)
svc_calibrated.fit(X_train_tfidf, y_train_aug)
print("    Treinado!")

# 2. SGDClassifier (segundo melhor: 0.75019)
print("\n--- SGDClassifier ---")
sgd = SGDClassifier(
    loss='modified_huber',  # Permite predict_proba
    alpha=1e-4,
    penalty='l2',
    max_iter=1000,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)
sgd.fit(X_train_tfidf, y_train_aug)
print("    Treinado!")

# 3. LogisticRegression (terceiro: 0.72935)
print("\n--- LogisticRegression ---")
lr = LogisticRegression(
    C=1.0,
    solver='lbfgs',
    max_iter=1000,
    class_weight='balanced',
    multi_class='multinomial',
    random_state=SEED,
    n_jobs=-1
)
lr.fit(X_train_tfidf, y_train_aug)
print("    Treinado!")

In [ ]:
# ==== VALIDAÇÃO CRUZADA ====
print("\n--- Cross-Validation (5-fold) ---")

# Usar dados originais para CV (sem augmentation) para avaliação justa
X_train_orig = train['text'].apply(preprocess)
y_train_orig = train['label'].values
X_train_orig_tfidf = tfidf.transform(X_train_orig)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Treinar modelos no dataset original para CV justa
svc_cv = CalibratedClassifierCV(LinearSVC(C=0.5, class_weight='balanced', dual='auto', random_state=SEED), cv=3)
sgd_cv = SGDClassifier(loss='modified_huber', alpha=1e-4, class_weight='balanced', random_state=SEED)
lr_cv = LogisticRegression(C=1.0, class_weight='balanced', multi_class='multinomial', random_state=SEED)

svc_scores = cross_val_score(svc_cv, X_train_orig_tfidf, y_train_orig, cv=cv, scoring='f1_macro')
sgd_scores = cross_val_score(sgd_cv, X_train_orig_tfidf, y_train_orig, cv=cv, scoring='f1_macro')
lr_scores = cross_val_score(lr_cv, X_train_orig_tfidf, y_train_orig, cv=cv, scoring='f1_macro')

print(f"\n    LinearSVC:     {svc_scores.mean():.5f} ± {svc_scores.std():.5f}")
print(f"    SGDClassifier: {sgd_scores.mean():.5f} ± {sgd_scores.std():.5f}")
print(f"    LogisticReg:   {lr_scores.mean():.5f} ± {lr_scores.std():.5f}")

In [ ]:
# ==== ENSEMBLE PREDICTIONS ====
print("\n[7/7] Gerando predições ensemble...")

# Predições individuais
pred_svc = svc_calibrated.predict(X_test_tfidf)
pred_sgd = sgd.predict(X_test_tfidf)
pred_lr = lr.predict(X_test_tfidf)

# Probabilidades
proba_svc = svc_calibrated.predict_proba(X_test_tfidf)
proba_sgd = sgd.predict_proba(X_test_tfidf)
proba_lr = lr.predict_proba(X_test_tfidf)

# ==== ESTRATÉGIAS DE ENSEMBLE ====

# 1. Hard Voting (maioria)
pred_hard = []
for i in range(len(test)):
    votes = [pred_svc[i], pred_sgd[i], pred_lr[i]]
    pred_hard.append(Counter(votes).most_common(1)[0][0])
pred_hard = np.array(pred_hard)

# 2. Soft Voting (média das probabilidades)
proba_avg = (proba_svc + proba_sgd + proba_lr) / 3
pred_soft = np.argmax(proba_avg, axis=1)

# 3. Weighted Voting (pesos baseados no score público)
# LinearSVC: 0.77885, SGD: 0.75019, LR: 0.72935
weights = np.array([0.77885, 0.75019, 0.72935])
weights = weights / weights.sum()  # Normalizar

proba_weighted = (proba_svc * weights[0] + proba_sgd * weights[1] + proba_lr * weights[2])
pred_weighted = np.argmax(proba_weighted, axis=1)

print("\n--- Comparação das estratégias ---")
print(f"Hard Voting - distribuição:  {dict(Counter(pred_hard))}")
print(f"Soft Voting - distribuição:  {dict(Counter(pred_soft))}")
print(f"Weighted    - distribuição:  {dict(Counter(pred_weighted))}")
print(f"LinearSVC   - distribuição:  {dict(Counter(pred_svc))}")

In [ ]:
# ==== CRIAR MÚLTIPLAS SUBMISSÕES ====
print("\n--- Gerando submissões ---")

# 1. LinearSVC puro (augmented) - provavelmente o melhor
submission_svc = pd.DataFrame({
    'id': test['id'],
    'label': pred_svc
})
submission_svc.to_csv('/kaggle/working/submission_linearsvc_aug.csv', index=False)
print("    submission_linearsvc_aug.csv")

# 2. Weighted Ensemble
submission_weighted = pd.DataFrame({
    'id': test['id'],
    'label': pred_weighted
})
submission_weighted.to_csv('/kaggle/working/submission_weighted_ensemble.csv', index=False)
print("    submission_weighted_ensemble.csv")

# 3. Soft Voting
submission_soft = pd.DataFrame({
    'id': test['id'],
    'label': pred_soft
})
submission_soft.to_csv('/kaggle/working/submission_soft_ensemble.csv', index=False)
print("    submission_soft_ensemble.csv")

# PRINCIPAL: usar weighted (melhor teórico)
submission_weighted.to_csv('/kaggle/working/submission.csv', index=False)

print("\n" + "="*60)
print("CONCLUÍDO - submission.csv = Weighted Ensemble")
print("="*60)
print("\nArquivos gerados:")
print("  - submission.csv (weighted ensemble)")
print("  - submission_linearsvc_aug.csv (LinearSVC + augmentation)")
print("  - submission_weighted_ensemble.csv")
print("  - submission_soft_ensemble.csv")

In [ ]:
# ==== ANÁLISE FINAL ====
print("\n--- Resumo Final ---")

print(f"\nDados de treino: {len(train)} → {len(train_aug)} (augmented)")
print(f"Dados de teste: {len(test)}")

print("\nModelos treinados:")
print("  1. LinearSVC (C=0.5, balanced)")
print("  2. SGDClassifier (modified_huber, balanced)")
print("  3. LogisticRegression (multinomial, balanced)")

print("\nTécnicas de Augmentation:")
print("  - Synonym replacement (termos médicos)")
print("  - Random word swap")
print("  - Random word deletion")
print("  - Noise injection (typos)")
print("  - Sentence shuffle")

print("\nEstrategias de Ensemble:")
print("  - Hard Voting (maioria)")
print("  - Soft Voting (média proba)")
print("  - Weighted Voting (pesos por score)")

print("\n✅ Pronto para submissão!")